In [1]:
from DefSD import *
from astropy.analytic_functions import blackbody
from astropy.convolution import convolve, Gaussian1DKernel, Box1DKernel



In [26]:
def finalFit(i, spec,lambd,ListLines,ListGal,spIvar,iterr, limResol):
    lineP = lineProfile2(i,spec,lambd,ListLines,ListGal, limResol)   #  Table of line profile indexes upper and lower 
    linePaux=lineP['lambda'].data     
    alslk = lineP['lambda'][0]
         #    it obtain the number of lines overlayed in a profile
    for c in range(1,len(linePaux)):
        if (linePaux[c]==alslk) & (alslk!=0):
            linePaux[c-1] = 2
            linePaux[c] = 2
        else:
            linePaux[c] = 1
        alslk = lineP['lambda'][c]
        
    f1 = fitting.LevMarLSQFitter()
    lineStdDev = 3.5    #   default standard deviation
    condit = True #   esto es mejor quitarlo
    while condit:    #   esto es mejor quitarlo
        Gaus=[]      #    array of models
        for x in range(len(ListLines)):       #     for each line profile it is created a gaussian model 
            if lineP['inf'][x]==0:            
                Gaus.append(models.Gaussian1D(amplitude=0,mean=ListLines['LAMBDA VAC ANG'][x],stddev=0))
                      #    it didn't model a profile for this line
            else:
                #v = np.where((lambd >= ListLines['LAMBDA VAC ANG'][x]-limResol) & (lambd <= ListLines['LAMBDA VAC ANG'][x]+limResol))
                      #    estimate the wavelenghts for the center of the line
                #innaaaads = v[0][0]
                innaaaads = lineP['lambda'][x]
                ampMax= ListGal[i][innaaaads]-spec[innaaaads]   #   max amplitude of the profile
                nLines = 1 #linePaux[x]                        #   obtain the number of overlaying line profiles 
                f = FWHM(lambd[lineP['inf'][x]:lineP['sup'][x]], ListGal[i][lineP['inf'][x]:lineP['sup'][x]])
                      #    calculate the FWHM of the profile
                coasi = 0
                while len(f)==0:
                    coasi = coasi + 1
                    f = FWHM(lambd[(lineP['inf'][x]-coasi):(lineP['sup'][x]+coasi)], ListGal[i][(lineP['inf'][x]-coasi):(lineP['sup'][x]+coasi)])
                           #  calculate FWHM so often as it is found a non profile spectrum
                print f
                stdLine = (1/2.3548)*f[0]/nLines     #   estimate the observed standard deviation
                lineWidthBase = (lambd[lineP['sup'][x]] - lambd[lineP['inf'][x]])/(2*nLines)
                      ##   calculate the observed line width at the base of the profile
                Gaus.append(models.Gaussian1D(amplitude=ampMax,mean=ListLines['LAMBDA VAC ANG'][x],stddev=stdLine, bounds={'mean':(ListLines['LAMBDA VAC ANG'][x]-lineWidthBase/(2*nLines), ListLines['LAMBDA VAC ANG'][x]+lineWidthBase/(2*nLines)),'stddev':(0.8*stdLine, 1.5*stdLine)}))
        condit = False  #   esto es mejor quitarlo
    sum_Gaussian=Gaus[0]
    for ooasia in range(1,len(Gaus)):
            #  sum of all the models respective to all the wavelenghts 
        sum_Gaussian=sum_Gaussian + Gaus[ooasia]
    gaussian_fit = f1(sum_Gaussian, lambd, ListGal[i]-spec, maxiter=iterr)
    Graph=[]
    Graph.append(gaussian_fit)   
    return Graph, lineP, spec     ##  return the fitting to both models and line profiles wavelenghts


In [27]:
def lineProfile2(i,spec, lambd,ListLines,ListGal, limResoluc):
    linePr = Table(names=('lambda', 'inf', 'sup'), dtype=('i5','i5', 'i5'))
    for c in range(0, len(ListLines)):
        lambdLine=ListLines['LAMBDA VAC ANG'][c]          #Líneas obtenidas de la base de datos Astroquery
        v = np.where((lambd >= lambdLine-2*limResoluc) & (lambd <= lambdLine+2*limResoluc))
        print v
        if len(v[0]) == 0:
            ind = 0
            l_inf = 0
            l_sup = 0
            iM=0
        else:
            ind = 1
            l_inf = v[0][0]-ind
            l_sup = v[0][-1]+ind
            slopeRight = spec[l_sup+1]-spec[l_sup]
            cooc = 0
            while cooc<1:
                l_sup=l_sup+1
                sloRaux = slopeRight
                slopeRight = spec[l_sup+1]-spec[l_sup]
                if slopeRight*sloRaux <= 0:
                    cooc = cooc + 1
            cooc = 0
            slopeLeft = spec[l_inf-1]-spec[l_inf]
            while cooc<1:
                l_inf=l_inf-1
                sloLaux = slopeLeft
                slopeLeft = spec[l_inf-1]-spec[l_inf]
                if slopeLeft*sloLaux <= 0:
                    cooc = cooc + 1
            l_sup=l_sup+1
            # Una vez encontrado el intervalo se busca el indice donde
            # tiene el máximo valor de intensidad
            indMaxInt = np.where(ListGal[i][l_inf:l_sup]==np.min(ListGal[i][l_inf:l_sup]))  # tomar el valor absoluto !!!
            indMax=indMaxInt[0][0]+l_inf
            iM = indMax
            
        linePr.add_row((iM, l_inf, l_sup))        
    return linePr

In [28]:
jj=0
resultListDT=Table.read('newObjSDSSFound.tbl', format='ascii')
name = resultListDT['ObjID'][0]
hdulist = fits.open('Exam_fits/'+name+'.fits')
head= hdulist[0].header
total= hdulist[1].data
linlam=np.exp(total['loglam']*np.log(10))          #  ocultar para leer desde archivo
lambd = linlam/(1+0.0001)
spectraFlux = total['flux']
Spectra = []
Spectra.append(spectraFlux)
Lambda=[]
Lambda.append(lambd)
ListaLineas=Table.read('LinesStar.txt', format='ascii')
spectraIvar = total['ivar']
lima = Lambda[jj][1]-Lambda[jj][0]

In [29]:
jj = 0
spssp = [100]*len(Spectra[0])
divisi = 10
lambdDiv = int(mt.floor(len(spssp)/divisi))
lambdDivArr = [lambdDiv*a for a in range(divisi)]
lambdDivArr[-1] = len(Spectra[0])
SG = []
for aa in range(len(lambdDivArr)-1):
    stdSpDiv = np.std(Spectra[0][lambdDivArr[aa]:lambdDivArr[aa+1]])
    #box_kernel = Box1DKernel(stdSpDiv)
    #smoothed_data_box = convolve(Spectra[0][lambdDivArr[aa]:lambdDivArr[aa+1]], box_kernel)
    #print stdSpDiv
    gauss_kernel = Gaussian1DKernel(0.05*stdSpDiv)
    meanSpDiv = np.mean(Spectra[0][lambdDivArr[aa]:lambdDivArr[aa+1]])
    smoothed_data_gauss = convolve(Spectra[0][lambdDivArr[aa]:lambdDivArr[aa+1]], gauss_kernel, boundary=u'extend',fill_value=meanSpDiv, normalize_kernel=True)
    SG.append(smoothed_data_gauss)
    spssp[lambdDivArr[aa]:lambdDivArr[aa+1]] = smoothed_data_gauss # smoothed_data_box
    #print "std: ", , "err: ", np.mean(SpectraIvar[0][lambdDivArr[aa]:lambdDivArr[aa+1]])
    #print Spectra[0][lambdDivArr[aa]:lambdDivArr[aa+1]]

#stdSp = np.std(Spectra[0])
#STD = [stdSp]*len(Spectra[0])
#spssp = spssp+STD

In [30]:
kLineP = lineProfile2(0, spssp, Lambda[0], ListaLineas, Spectra, lima)

(array([1882, 1883]),)
(array([3472, 3473]),)
(array([3495, 3496]),)
(array([3556, 3557]),)


In [31]:
spec2 = spssp
for ina in range(len(kLineP)):
    lInf = kLineP[ina]['inf']
    lSup = kLineP[ina]['sup']
    lDel = lSup-lInf
    if (lInf-1*lDel) >= 0 :
        lInf = lInf-1*lDel
    if (lSup+1*lDel) <= len(lambd) :
        lSup = lSup+1*lDel
    m1, b1 = linModel( Lambda[jj][lInf] , spec2[lInf] , Lambda[jj][lSup], spec2[lSup])
    lineLambd = Lambda[jj][lInf:lSup]
    lineIntCont = lines( lineLambd , m1, b1)
    spec2[lInf:lSup] = lineIntCont
SpectraCont=[]
SpectraCont.append(spec2)

In [35]:
%matplotlib
plt.plot(Lambda[jj],Spectra[jj], lw=0.5, c = 'k', label='Spectra')
plt.plot(Lambda[jj],spssp, lw=2, c = 'cyan', label='Spectra')
plt.plot(Lambda[jj],spec2, lw=2, c = 'g', label='Spectra')
lPPP = kLineP
#plt.plot(Lambda[jj],Spectra[jj]-spssp, lw=0.5, c = 'gray', label='Spectra')
for iasiao in range(len(lPPP)):
    plt.plot(Lambda[jj][lPPP['inf'][iasiao]:lPPP['sup'][iasiao]],Spectra[jj][lPPP['inf'][iasiao]:lPPP['sup'][iasiao]], lw=2.0, c = 'b')


Using matplotlib backend: Qt4Agg


In [32]:
ModelG1, lineProf, spec2 = finalFit(0, spec2, Lambda[jj],ListaLineas,Spectra, spectraIvar,2000, lima)
Modelos = []
Modelos.append(ModelG1)
LS=np.zeros([len(ListaLineas),10])
Tabla=Table(LS,names=('Amplitude','Wavelength','Sigma','Lambda Vac','Element','Flux','Flux_Err', 'FWHM', 'eqWidth', 'S/N>3'),
                    dtype=('f8','f8','f8','f8','S8','f8','f8','f8','f8','S5'))
Tabla['Lambda Vac']=ListaLineas['LAMBDA VAC ANG']
Tabla['Element']=ListaLineas['SPECTRUM']

AA=split_list(ModelG1[0].parameters, wanted_parts=len(ListaLineas))
for ii in range(len(ListaLineas)):
    if 3*np.mean(spectraIvar[lineProf[0]['inf']:lineProf[0]['sup']]) >= AA[ii][0]:
        Tabla['S/N>3'][ii] = str(False)
    else:
        Tabla['S/N>3'][ii] = str(True)
    Tabla['Amplitude'][ii],Tabla['Wavelength'][ii],Tabla['Sigma'][ii]=np.array([AA[ii][0], AA[ii][1], AA[ii][2]])
Tabla

(array([1882, 1883]),)
(array([3472, 3473]),)
(array([3495, 3496]),)
(array([3556, 3557]),)
[-12.21435547]
[-1.9609375  -7.87207031]
[-1.9609375  -7.87207031]
[-5.99023438]


Amplitude,Wavelength,Sigma,Lambda Vac,Element,Flux,Flux_Err,FWHM,eqWidth,S/N>3
float64,float64,float64,float64,string64,float64,float64,float64,float64,string40
-157.912190851,5894.24233532,-5.13940867061,5894.0,[O I],0.0,0.0,0.0,0.0,False
-53.0844037105,8500.52576787,-1.90926527055,8500.0,[O I],0.0,0.0,0.0,0.0,False
-87.3407639504,8544.57266232,-2.88288244392,8544.9,[O I],0.0,0.0,0.0,0.0,False
-79.1995296149,8664.77513177,-2.142246992,8665.01,[O I],0.0,0.0,0.0,0.0,False


In [33]:
%matplotlib
plt.plot(Lambda[jj], Spectra[jj]-SpectraCont[jj], c='k', lw=0.8, label='Model')
plt.plot(Lambda[jj], Modelos[jj][0](Lambda[jj]), c='yellow', lw=1)


Using matplotlib backend: Qt4Agg


In [10]:
kLineP

lambda,inf,sup
int32,int32,int32
1881,1867,1896
3473,3469,3478
3495,3487,3502
3556,3551,3561
